## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]
print(y_df.head())

  Attrition              Department
0       Yes                   Sales
1        No  Research & Development
2       Yes  Research & Development
3        No  Research & Development
4        No  Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data

X_data =['Education',
         'Age',
         'DistanceFromHome',
         'JobSatisfaction',
         'OverTime',
         'StockOptionLevel',
        'WorkLifeBalance',
         'YearsAtCompany',
         'YearsSinceLastPromotion',
         'NumCompaniesWorked']

# Create X_df using your selected columns

X_df = attrition_df[X_data]

# Show the data types for X_df

X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [5]:
# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X_df,y_df)

In [6]:
print(X_df.columns)

Index(['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
       'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'NumCompaniesWorked'],
      dtype='object')


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

X_train['OverTime'].value_counts()

,count
OverTime,
No,790
Yes,312


In [8]:
# Fit the encoder on the training data

OverTime_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
OverTime_encoder.fit(np.array(X_train['OverTime']).reshape(-1, 1))

# Transform the training and test data

X_train['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_train['OverTime']).reshape(-1, 1))
X_test['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_test['OverTime']).reshape(-1, 1))
X_train = X_train.drop(columns='OverTime')
X_test = X_test.drop(columns='OverTime')

# Check the data types

print(X_train.dtypes)
print(X_test.dtypes)

Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTimeEncoded            float64
dtype: object
Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTimeEncoded            float64
dtype: object


In [9]:
# Create a StandardScaler

scaler = StandardScaler()

# Fit the StandardScaler to the training data

X_scaler = scaler.fit(X_train)

# Scale the training and testing data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Create a OneHotEncoder for the Department column

department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data

department_encoder.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data

y_department_train = department_encoder.transform(np.array(y_train['Department']).reshape(-1, 1))
y_department_test = department_encoder.transform(np.array(y_test['Department']).reshape(-1, 1))

y_department_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [11]:
# Create a OneHotEncoder for the Attrition column

attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data

attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data

y_attrition_train = attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1, 1))
y_attrition_test = attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1, 1))

y_attrition_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [12]:
# Find the number of columns in the X training data

features = len(X_train.columns)

# Create the input layer

input = layers.Input((features,),name = 'input')

# Create at least two shared layers

shared1 = layers.Dense(64, activation='relu',name = 'shared1')(input)
shared2 = layers.Dense(128, activation='relu',name = 'shared2')(shared1)

In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer

department_hidden = layers.Dense(32, activation='relu',name = 'department_hidden')(shared2)

# Create the output layer

department_output = layers.Dense(y_department_train.shape[1], activation='sigmoid',name = 'department_output')(department_hidden)


In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer

attrition_hidden = layers.Dense(32, activation='relu',name = 'attrition_hidden')(shared2)

# Create the output layer

attrition_output = layers.Dense(y_attrition_train.shape[1], activation='sigmoid',name = 'attrition_output')(attrition_hidden)



In [15]:
# Create the model

model = Model(inputs = input, outputs={'department_output': department_output, 'attrition_output': attrition_output})

# Compile the model

model.compile(optimizer='adam',
              loss={
                  'department_output': 'categorical_crossentropy',
                  'attrition_output': 'binary_crossentropy'
              },
              metrics={
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              })
# Summarize the model

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model

fit_model = model.fit(X_train_scaled, {'department_output': y_department_train, 'attrition_output': y_attrition_train}, epochs=100)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - attrition_output_accuracy: 0.7977 - attrition_output_loss: 0.5835 - department_output_accuracy: 0.5391 - department_output_loss: 0.9170 - loss: 1.5009
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8400 - attrition_output_loss: 0.4250 - department_output_accuracy: 0.6758 - department_output_loss: 0.7491 - loss: 1.1741
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8400 - attrition_output_loss: 0.4004 - department_output_accuracy: 0.6511 - department_output_loss: 0.7585 - loss: 1.1587
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8790 - attrition_output_loss: 0.3400 - department_output_accuracy: 0.6568 - department_output_loss: 0.7451 - loss: 1.0851
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8731 - attrition_output_loss: 0.3550 - department_output_accuracy: 0.6663 - department_output_loss: 0.

In [17]:
# Evaluate the model with the testing data

results = model.evaluate(X_test_scaled,{'department_output': y_department_test, 'attrition_output': y_attrition_test})
results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8104 - attrition_output_loss: 1.4086 - department_output_accuracy: 0.5083 - department_output_loss: 3.0467 - loss: 4.4462  


[4.626735687255859,
 1.388494610786438,
 3.2972495555877686,
 0.8070651888847351,
 0.5]

In [18]:
# Print the accuracy for both department and attrition

print(f"Department predictions accuracy: {results[4]}")
print(f"Attrition predictions accuracy:{results[3]}")

Department predictions accuracy: 0.5
Attrition predictions accuracy:0.8070651888847351


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is a very common metric to use on data for multiple reasons. For one, it is a simple metric to communicate the model's performance. However, whether or not accuracy is the best metric for the data in this assignment is debatable. For the y_df column attrition, we are working with binary data. Binary data is great with accuracy metircs as the classification task is straight forward. We do not however know how well balanced this binary data is which is why I note this as debatable. I can however, see how well the accuracy score performed with a score of 0.807, and this leads me to believe that the accuracy metric for the attrition column is a great choice. Now for the department predictions we are less confident in this choice as the score was only a 0.5 and the data was not binary. Without knowing exactly how balanced or unbalanced the y_df department column is, we can only glean that the low score in the accuracy metric is due to a possible inbalance in the data and therefor indicating the model had low accuracy.

2. The output layers in both of the attrition output as well as the department output were working off of a hidden layer activation relu function and the output layers for both were sigmoid. Activation relu, or rectified linear unit, handles any negative input by putting out a zero in its place and is typically used in the hidden layers of neural networks. Now the activation function that was chosen for the output layers was the sigmoid funciton. For binary classification tasks this is ideal, and for attrition this was the best choice. Sigmoid outputs numbers into either a zero or a one, this is ideal also for binary as our attrition data is. In the deparment branch sigmoid has a smooth gradient which works well with training data because they aid in stable and efficient models.

3. A few ways that this model might be improved is to change the output layer from sigmoid function to softmax in order to better work with the non-binary data of the department column. Softmax can handle multiple classes as well as better probability distribution across all classes. Another way that this model might be improved is by increasing model complexity. So for example, we could add hidden layers and move up from 32 to 64. We could also try different activation functions. Another option to improve our model would be in preprocessing, we could work to make sure our data is balanced and make improvements on the unbalance such as oversampling or undersampling. I can not elaborate on the best methods in which to handle the inbalance because I have not analyzed the balance or unbalance of the data to decide the best plan.